<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f5426dd3461d43b2755ba202528074f45bbef5581add31fbf0b2b79153450210
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-31 10:13:26
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 17
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.48 L
Current:  1.46 C
-------------------
Today PnL: -453.00 (-0.0%)
Current PnL: -18.47 L (-12.1%)
CY Booked + Current PnL: -6.34 L (-4.15%)
-------------------
Total profit:  2.88 L
Total loss:  -21.35 L
-------------------
Total Booked + Current PnL: 20.64 L (16.43%)
Total Booked PnL: 39.11 L (31.13%)
Curr Year Booked PnL: 12.13 L (8.32%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.54 L (58.7%)
Deployed:  1.26 C
Current:  1.46 C
CAGR/XIRR %: 8.82%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1533.0,-7.28,68.0,X-LC,4.91,231738.0,21372.0,7416.0,-0.20,10.16,3.20,13.68,37.0,2.88,1.61,28.36,XY25,NTT,REFINERIES
12,BANKINDIA,149.0,-15.53,72.0,H-MC,17.67,216981.0,37173.0,12173.0,1.61,20.67,5.61,27.45,88.0,3.05,1.50,61.81,XR,NTT,BANKS
79,TTKPRESTIG,770.0,105.55,57.0,M-SC,1.47,88543.0,-12234.0,12325.0,-0.70,-12.14,13.92,0.09,245.0,-0.99,0.61,16.33,OX40N,NTT,DURABLES
42,ITC,452.0,-35.81,67.0,X-LC,1.20,207898.0,7760.0,13139.0,1.53,3.88,6.32,10.44,4.0,0.59,1.44,9.66,X40,NTT,FMCG
80,UNIONBANK,163.0,-6.98,64.0,M-LC,9.94,168901.0,28061.0,16434.0,4.38,19.92,9.73,31.59,66.0,1.71,1.17,51.69,XY24,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,UNITDSPR,1644.0,-3.95,76.0,X-LC,4.53,251705.0,17561.0,31060.0,4.93,7.50,12.34,20.77,86.0,0.57,1.74,14.94,X40N,NTT,BREWERIES
80,UNIONBANK,163.0,-6.98,64.0,M-LC,9.94,168901.0,28061.0,16434.0,4.38,19.92,9.73,31.59,66.0,1.71,1.17,51.69,XY24,NTT,BANKS
70,STARHEALTH,761.0,21.02,61.0,H-SC,13.49,269797.0,-11326.0,144180.0,3.10,-4.03,53.44,47.26,144.0,-0.08,1.87,44.55,XY24,NTT,INSURANCE
56,RAJOOENG,143.1,-39.71,53.0,H-SC,16.73,91290.0,-11210.0,51807.0,2.76,-10.94,56.75,39.61,114.0,-0.22,0.63,11.49,AR,ATH,MISC
55,RAJESHEXPO,518.0,1689.14,52.0,L-SC,3.63,48832.0,-88345.0,88440.0,2.21,-64.40,181.11,0.07,267.0,-1.00,0.34,21.45,OX40N,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BANDHANBNK,400.00,-13.31,45.0,H-SC,3.50,217670.0,-60889.0,318321.0,-4.77,-21.86,146.24,92.42,151.0,-0.19,1.51,25.80,XY24,NTT,BANKS
62,SAIL,228.00,49.16,59.0,M-MC,10.23,236000.0,11038.0,156610.0,-2.49,4.91,66.36,74.53,192.0,0.07,1.64,38.74,XY24,BTT,STEEL
22,DABUR,735.00,-11.76,40.0,X-MC,3.45,238620.0,-9448.0,117854.0,-1.90,-3.81,49.39,43.70,102.0,-0.08,1.65,10.09,XY24,BTT,FMCG
84,VBL,671.64,-15.68,57.0,X-LC,7.96,303816.0,-12026.0,124686.0,-1.86,-3.81,41.04,35.67,5.0,-0.10,2.11,9.52,X40N,ATH,FMCG
54,QUESS,986.00,-46.56,46.0,X-SC,37.80,53151.0,-11855.0,162780.0,-1.36,-18.24,306.26,232.18,198.0,-0.07,0.37,3.67,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,WIPRO,420.0,-14.25,46.0,M-LC,5.28,150474.0,-471.0,109921.0,0.32,-0.31,73.05,72.51,53.0,-0.0,1.04,5.69,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TTKPRESTIG,770.0,105.55,57.0,M-SC,1.47,88543.0,-12234.0,12325.0,-0.70,-12.14,13.92,0.09,245.0,-0.99,0.61,16.33,OX40N,NTT,DURABLES
46,KANSAINER,340.0,-67.42,51.0,H-SC,2.85,225135.0,-44532.0,80868.0,-0.14,-16.51,35.92,13.47,138.0,-0.55,1.56,14.47,XY24,NTT,PAINTS
18,CERA,9475.0,-22.20,54.0,H-SC,2.31,141876.0,-34027.0,76046.0,-0.24,-19.34,53.60,23.89,149.0,-0.45,0.98,22.53,OX40N,NTT,CERAMICS
74,TATAELXSI,9161.0,-22.37,54.0,H-MC,6.13,105070.0,-26765.0,68989.0,-0.17,-20.30,65.66,32.03,98.0,-0.39,0.73,17.69,OX40N,NTT,IT
68,SIS,528.0,2010.36,42.0,H-SC,3.29,85157.0,-25875.0,49485.0,-0.91,-23.30,58.11,21.26,156.0,-0.52,0.59,15.06,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.0,-15.53,72.0,H-MC,17.67,216981.0,37173.0,12173.0,1.61,20.67,5.61,27.45,88.0,3.05,1.5,61.81,XR,NTT,BANKS


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,SAIL,228.00,49.16,59.0,M-MC,10.23,236000.0,11038.0,156610.0,-2.49,4.91,66.36,74.53,192.0,0.07,1.64,38.74,XY24,BTT,STEEL
57,RECLTD,446.00,47.87,55.0,M-LC,5.18,207790.0,4620.0,37506.0,-0.13,2.27,18.05,20.74,55.0,0.12,1.44,9.29,XY25,NTT,FINANCE
32,HINDZINC,730.22,27.88,49.0,M-LC,11.29,208278.0,3202.0,109367.0,0.25,1.56,52.51,54.89,52.0,0.03,1.44,26.03,X5K,ATH,METALS
47,KPIGREEN,731.05,11.71,70.0,H-SC,5.24,133094.0,7797.0,51135.0,-1.17,6.22,38.42,47.03,141.0,0.15,0.92,66.65,MH,ATH,POWER
80,UNIONBANK,163.00,-6.98,64.0,M-LC,9.94,168901.0,28061.0,16434.0,4.38,19.92,9.73,31.59,66.0,1.71,1.17,51.69,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,149.00,-15.53,72.0,H-MC,17.67,216981.0,37173.0,12173.0,1.61,20.67,5.61,27.45,88.0,3.05,1.50,61.81,XR,NTT,BANKS
86,WHIRLPOOL,2270.00,-37.38,65.0,M-SC,4.05,129720.0,14922.0,79116.0,-0.77,13.00,60.99,81.92,223.0,0.19,0.90,53.57,XR,NTT,DURABLES
47,KPIGREEN,731.05,11.71,70.0,H-SC,5.24,133094.0,7797.0,51135.0,-1.17,6.22,38.42,47.03,141.0,0.15,0.92,66.65,MH,ATH,POWER
37,INDIAMART,4810.62,-53.52,54.0,H-SC,6.51,129956.0,6620.0,125005.0,-0.71,5.37,96.19,106.72,119.0,0.05,0.90,30.00,AR,ATH,MISC
32,HINDZINC,730.22,27.88,49.0,M-LC,11.29,208278.0,3202.0,109367.0,0.25,1.56,52.51,54.89,52.0,0.03,1.44,26.03,X5K,ATH,METALS


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TATAMOTORS,1065.00,-52.51,20.0,X-LC,22.16,149719.0,-126804.0,236870.0,0.27,-45.86,158.21,39.81,54.0,-0.54,1.04,5.53,XY24,NTT,AUTO
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.35,88893.0,-39777.0,85071.0,-0.09,-30.91,95.70,35.20,219.0,-0.47,0.62,2.03,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,35.0,X-MC,2.35,88215.0,-2463.0,17370.0,-0.66,-2.72,19.69,16.44,101.0,-0.14,0.61,16.91,X40,ATH,PHARMA
45,JSWINFRA,342.00,-20.89,36.0,X-MC,5.12,198450.0,-1897.0,32407.0,-0.47,-0.95,16.33,15.23,178.0,-0.06,1.38,31.00,X40N,NTT,REALTY
24,DMART,5391.45,-17.64,37.0,X-LC,9.36,100757.0,-1829.0,28635.0,0.89,-1.78,28.42,26.13,38.0,-0.06,0.70,23.33,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
42,ITC,452.00,-35.81,67.0,X-LC,1.20,207898.0,7760.0,13139.0,1.53,3.88,6.32,10.44,4.0,0.59,1.44,9.66,X40,NTT,FMCG
67,SIEMENS,4671.50,-2.18,49.0,H-LC,1.23,157065.0,-29030.0,76506.0,0.46,-15.60,48.71,25.51,15.0,-0.38,1.09,15.74,AR,ATH,ELECTRICAL
53,PGHH,17907.65,-30.55,38.0,X-MC,2.01,200865.0,45.0,67752.0,-0.16,0.02,33.73,33.76,80.0,0.00,1.39,5.22,X40,ATH,FMCG
29,HAVELLS,2069.16,-5.47,50.0,X-MC,2.87,315567.0,-12424.0,118969.0,-0.24,-3.79,37.70,32.48,92.0,-0.10,2.19,7.20,X40,ATH,ELECTRICAL
85,VOLTAS,1530.00,1.02,51.0,X-MC,4.52,211860.0,20118.0,17648.0,-0.52,10.49,8.33,19.69,99.0,1.14,1.47,18.00,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.35,88893.0,-39777.0,85071.0,-0.09,-30.91,95.70,35.20,219.0,-0.47,0.62,2.03,X40,NTT,FOOTWEAR
54,QUESS,986.00,-46.56,46.0,X-SC,37.80,53151.0,-11855.0,162780.0,-1.36,-18.24,306.26,232.18,198.0,-0.07,0.37,3.67,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.42,42.0,X-MC,6.75,82040.0,-1470.0,21503.0,0.18,-1.76,26.21,23.98,82.0,-0.07,0.57,3.96,X40,ATH,APPARELS
10,BAJAJHFL,181.50,-14.27,47.0,X-MC,5.94,180580.0,-19926.0,117088.0,-0.55,-9.94,64.84,48.45,90.0,-0.17,1.25,4.13,X40N,ATH,FINANCE
21,COLPAL,3767.14,-3.66,48.0,X-MC,7.49,224910.0,-38455.0,151814.0,-0.27,-14.60,67.50,43.04,84.0,-0.25,1.56,4.42,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,986.00,-46.56,46.0,X-SC,37.80,53151.0,-11855.0,162780.0,-1.36,-18.24,306.26,232.18,198.0,-0.07,0.37,3.67,XY24,NTT,MISC
52,PAGEIND,51769.93,-27.42,42.0,X-MC,6.75,82040.0,-1470.0,21503.0,0.18,-1.76,26.21,23.98,82.0,-0.07,0.57,3.96,X40,ATH,APPARELS
58,RELAXO,1176.00,-39.68,44.0,X-SC,4.66,82184.0,-62976.0,138907.0,-0.93,-43.38,169.02,52.31,136.0,-0.45,0.57,10.01,X40N,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.65,35.0,X-MC,2.35,88215.0,-2463.0,17370.0,-0.66,-2.72,19.69,16.44,101.0,-0.14,0.61,16.91,X40,ATH,PHARMA
13,BATAINDIA,2096.00,-40.58,28.0,X-SC,5.35,88893.0,-39777.0,85071.0,-0.09,-30.91,95.70,35.20,219.0,-0.47,0.62,2.03,X40,NTT,FOOTWEAR


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TCS,4389.97,-27.72,55.0,X-LC,10.58,287537.0,-58419.0,125107.0,0.78,-16.89,43.51,19.28,1.0,-0.47,1.99,6.30,X40,ATH,IT
40,INFY,2275.00,-18.52,54.0,X-LC,5.49,317498.0,4610.0,167067.0,-0.21,1.47,52.62,54.87,3.0,0.03,2.20,9.88,X40,BTT,IT
42,ITC,452.00,-35.81,67.0,X-LC,1.20,207898.0,7760.0,13139.0,1.53,3.88,6.32,10.44,4.0,0.59,1.44,9.66,X40,NTT,FMCG
84,VBL,671.64,-15.68,57.0,X-LC,7.96,303816.0,-12026.0,124686.0,-1.86,-3.81,41.04,35.67,5.0,-0.10,2.11,9.52,X40N,ATH,FMCG
1,ABB,7934.00,-39.57,54.0,H-LC,5.43,252528.0,-9091.0,128309.0,-0.34,-3.47,50.81,45.57,7.0,-0.07,1.75,7.45,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,6994.44,57.0,L-SC,8.24,77837.0,-15973.0,92587.0,0.03,-17.03,118.95,81.67,269.0,-0.17,0.54,50.73,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-45.67,49.0,L-SC,28.33,81235.0,-32314.0,72364.0,0.47,-28.46,89.08,35.27,268.0,-0.45,0.56,98.83,XR,NTT,HOTELS
50,MASFIN,398.61,-18.70,52.0,H-SC,8.75,92250.0,-5730.0,27334.0,-0.29,-5.85,29.63,22.05,152.0,-0.21,0.64,34.09,XR,ATH,FINANCE
37,INDIAMART,4810.62,-53.52,54.0,H-SC,6.51,129956.0,6620.0,125005.0,-0.71,5.37,96.19,106.72,119.0,0.05,0.90,30.00,AR,ATH,MISC
86,WHIRLPOOL,2270.00,-37.38,65.0,M-SC,4.05,129720.0,14922.0,79116.0,-0.77,13.00,60.99,81.92,223.0,0.19,0.90,53.57,XR,NTT,DURABLES


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,KPIGREEN,731.05,11.71,70.0,H-SC,5.24,133094.0,7797.0,51135.0,-1.17,6.22,38.42,47.03,141.0,0.15,0.92,66.65,MH,ATH,POWER
82,VAIBHAVGBL,521.00,76.31,73.0,H-SC,2.20,152641.0,-30134.0,141208.0,-1.23,-16.49,92.51,60.77,125.0,-0.21,1.06,36.71,XR,NTT,JEWELLERY
86,WHIRLPOOL,2270.00,-37.38,65.0,M-SC,4.05,129720.0,14922.0,79116.0,-0.77,13.00,60.99,81.92,223.0,0.19,0.90,53.57,XR,NTT,DURABLES
63,SAMMAANCAP,326.00,-197.73,76.0,M-SC,30.25,169542.0,20322.0,123850.0,2.17,13.62,73.05,96.62,208.0,0.16,1.18,83.48,XY25,NTT,FINANCE
5,ANGELONE,3033.00,15.82,61.0,X-SC,5.70,204672.0,13666.0,44025.0,-0.68,7.15,21.51,30.21,157.0,0.31,1.42,29.53,X40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.68
1,25,43.97
2,50,74.93


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.05
LC    32.42
MC    24.50
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     29.99
X40      16.21
XY25     12.05
XR       11.45
X40N     10.71
AR        7.82
OX40N     6.52
X200      1.79
X5K       1.44
SR        1.07
MH        0.92
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.40
X-LC    21.70
X-MC    18.67
M-SC    12.69
M-LC     5.09
H-LC     4.63
X-SC     4.52
H-MC     3.87
M-MC     1.64
L-SC     1.44
L-LC     1.00
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.45,-3.34,37.06
IT,12.47,-16.78,79.40
FINANCE,12.13,-8.06,57.04
MISC,6.85,-16.29,80.73
ELECTRICAL,5.80,-8.03,46.33
BANKS,5.61,-5.79,60.49
PAINTS,5.49,-16.62,34.00
INSURANCE,3.84,0.32,36.85
AUTO,2.76,-44.16,104.32


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3245333.0,22
XR,1234418.0,14
AR,1158685.0,9
X40,817835.0,12
X40N,615922.0,9
OX40N,542343.0,9
XY25,445319.0,8
SR,267214.0,2
X5K,109367.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3311300.0,24
M-SC,1211471.0,15
X-MC,1183117.0,14
X-LC,1009659.0,13
X-SC,529897.0,6
M-LC,273228.0,4
H-LC,271342.0,3
H-MC,257696.0,3
L-SC,253391.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1156233.0      6
           AR         830760.0      5
           XR         763923.0      7
M-SC       XY24       695168.0      6
X-MC       XY24       578310.0      4
X-LC       X40        446914.0      5
           XY24       303264.0      2
X-MC       X40        285850.0      6
X-SC       X40N       282046.0      4
H-SC       SR         267214.0      2
           OX40N      242035.0      3
H-LC       AR         204815.0      2
X-LC       X40N       184381.0      3
H-MC       XY24       176534.0      1
X-MC       XY25       169462.0      2
L-SC       XR         164951.0      2
X-SC       XY24       162780.0      1
M-MC       XY24       156610.0      1
X-MC       X40N       149495.0      2
M-SC       OX40N      142879.0      4
           XR         126464.0      2
           XY25       123850.0      1
           AR         123110.0      2
M-LC       XR         109921.0      1
           X5K        109367.0      1
L-SC       OX40N       88440.0      1
X-SC       X40         85071.0      1
X-LC       XY25        75100.0      3
H-MC       OX40N       68989.0      1
H-LC       X200        66527.0      1
L-MC       XR          56986.0      1
H-SC       MH          51135.0      1
L-LC       XY25        39401.0      1
M-LC       XY25        37506.0      1
           XY24        16434.0      1
H-MC       XR          12173.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 32.0 seconds
